# Problem 666 - Polymorphic Bacteria

Members of a species of bacteria occur in two different types: $\alpha$ and $\beta$. Individual bacteria are capable of multiplying and mutating between the types according to the following rules:
<ul><li>Every minute, each individual will simultaneously undergo some kind of transformation.</li>
<li>Each individual $A$ of type $\alpha$ will, independently, do one of the following (at random with equal probability):
<ul><li>clone itself, resulting in a new bacterium of type $\alpha$ (alongside $A$ who remains)</li>
<li>split into 3 new bacteria of type $\beta$ (replacing $A$)</li>
</ul></li>

<li>Each individual $B$ of type $\beta$ will, independently, do one of the following (at random with equal probability):
<ul><li>spawn a new bacterium of type $\alpha$ (alongside $B$ who remains)</li>
<li>die</li>
</ul></li></ul><p>
If a population starts with a single bacterium of type $\alpha$, then it can be shown that there is a 0.07243802 probability that the population will eventually die out, and a 0.92756198 probability that the population will last forever. These probabilities are given rounded to 8 decimal places.
</p>
<p>
Now consider another species of bacteria, $S_{k,m}$ (where $k$ and $m$ are positive integers), which occurs in $k$ different types $\alpha_i$ for $0\le i\lt k$. The rules governing this species' lifecycle involve the sequence $r_n$ defined by:
</p>
<ul style="list-style-type:none;"><li>$r_0 = 306$</li>
<li>$r_{n+1} = r_n^2 \bmod 10\,007$</li>
</ul><p>
Every minute, for each $i$, each bacterium $A$ of type $\alpha_i$ will independently choose an integer $j$ uniformly at random in the range $0\le j\lt m$. What it then does depends on $q = r_{im+j} \bmod 5$:</p>
<ul><li>If $q=0$, $A$ dies.</li>
<li>If $q=1$, $A$ clones itself, resulting in a new bacterium of type $\alpha_i$ (alongside $A$ who remains).</li>
<li>If $q=2$, $A$ mutates, changing into type $\alpha_{(2i) \bmod k}$.</li>
<li>If $q=3$, $A$ splits into 3 new bacteria of type $\alpha_{(i^2+1) \bmod k}$ (replacing $A$).</li>
<li>If $q=4$, $A$ spawns a new bacterium of type $\alpha_{(i+1) \bmod k}$ (alongside $A$ who remains).</li>
</ul><p>
In fact, our original species was none other than $S_{2,2}$, with $\alpha=\alpha_0$ and $\beta=\alpha_1$.
</p>
<p>
Let $P_{k,m}$ be the probability that a population of species $S_{k,m}$, starting with a single bacterium of type $\alpha_0$, will eventually die out. So $P_{2,2} = 0.07243802$. You are also given that $P_{4,3} = 0.18554021$ and $P_{10,5} = 0.53466253$, all rounded to 8 decimal places.
</p>
<p>
Find $P_{500,10}$, and give your answer rounded to 8 decimal places.
</p>

In [28]:
from functools import cache
from collections import defaultdict
import sympy as sp
import numpy as np
from tqdm import tqdm

In [29]:
@cache
def r(n):
    if n == 0:
        return 306

    return r(n-1)**2 % 10_007

In [30]:
@cache
def q(i, m):
    '''
    Finds distribution of q
    '''
    q_dist = defaultdict(int)
    r_set = set([r(n) for n in range(10007)])
        
    for j in range(m):
        if i*m + j == 0:
            q_dist[306 % 5] += 1
        else:
            q_dist[r(i*m + j) % 5] += 1
            
    for t in q_dist:
        q_dist[t] = q_dist[t] / m

    return q_dist

In [34]:
def F(k, m, a):
    b = []
    for i in range(k):
        q0, q1, q2, q3, q4 = q(i, m)[0], q(i, m)[1], q(i, m)[2], q(i, m)[3], q(i, m)[4]
        b.append(
            a[i]
            - q0
            - q1 * a[i]**2
            - q2 * a[(2*i) % k]
            - q3 * a[(i**2 + 1) % k]**3
            - q4 * a[(i + 1) % k] * a[i]
        )

    return b

def J_F(k, m, a):
    J = np.zeros((k, k))

    for i in range(k):
        q0, q1, q2, q3, q4 = q(i, m)[0], q(i, m)[1], q(i, m)[2], q(i, m)[3], q(i, m)[4]

        J[i][i] = (
            1
            - 2 * q1 * a[i]
            - q4 * a[(i + 1) % k]
        )

        J[i][(2*i)%k] += -q2

        J[i][(i**2 + 1)%k] += -3 * q3 * a[(i**2 + 1) % k]**2

        J[i][(i + 1) % k] += -q4 * a[i]

    return J


In [33]:
def P(k, m, max_iters=10000, tol=1e-8):
    a = np.full(k, 0.5)

    for _ in tqdm(range(max_iters)):
        F_val = np.array(F(k, m, a))
        J = J_F(k, m, a)

        try:
            delta = np.linalg.solve(J, F_val)
        except np.linalg.LinAlgError:
            print("Jacobian is singular or ill-conditioned. Aborting.")
            break

        a_new = a - delta

        if np.linalg.norm(delta, ord=2) < tol:
            break

        a = a_new

    return a.tolist()

In [36]:
P(500,10)

  0%|                                                                                | 6/10000 [00:00<22:22,  7.45it/s]


[0.48023167580091347,
 0.3765885046099761,
 0.16566732115836025,
 0.2270759754935653,
 0.1323895032968863,
 0.23030645971586264,
 0.3448144281554122,
 0.2561167234686506,
 0.2942788858538912,
 0.47739843715374236,
 0.0006524580720109159,
 0.279324819163569,
 0.4240391629777508,
 0.22809873940102324,
 0.12737711692782724,
 0.07046411085155098,
 0.16212619044012933,
 0.32145893789998214,
 0.32014990047667147,
 0.1334673964894131,
 0.2409417853767567,
 0.48686234664810757,
 0.5046902342415033,
 0.8114758535102861,
 0.2291092030786617,
 0.07198661708608944,
 0.21419520237170203,
 0.2713677455672723,
 0.27009032150490314,
 0.2896858843834919,
 0.3170602462940018,
 0.28819158302755243,
 0.28347277711403546,
 0.5950024196032373,
 0.32071717246704623,
 0.5071209079792449,
 0.4098380524641517,
 0.22401449633825382,
 0.21858969258262156,
 0.299091424417829,
 0.3168819643258488,
 0.32683519420759066,
 0.5200304419319585,
 0.2828238896926147,
 0.2432755581743573,
 0.37522855053906884,
 0.424392295